In [1]:
#clip_multiple-raster
import geopandas as gpd
import rasterio as rio
from rasterio.mask import mask as rmask
import os

# Path to the shapefile (Region of Interest)
roi = r"G:\Manjar\github\boundary\delhi.shp"

# List of raster paths
rasters_path = [
    r"G:\Manjar\github\rasterData\LC08_L2SP_146040_20241018_20241028_02_T1_SR_B1.TIF",
    r"G:\Manjar\github\rasterData\LC08_L2SP_146040_20241018_20241028_02_T1_SR_B2.TIF",
    r"G:\Manjar\github\rasterData\LC08_L2SP_146040_20241018_20241028_02_T1_SR_B3.TIF",
    r"G:\Manjar\github\rasterData\LC08_L2SP_146040_20241018_20241028_02_T1_SR_B4.TIF",
    r"G:\Manjar\github\rasterData\LC08_L2SP_146040_20241018_20241028_02_T1_SR_B5.TIF",
    r"G:\Manjar\github\rasterData\LC08_L2SP_146040_20241018_20241028_02_T1_SR_B6.TIF",
    r"G:\Manjar\github\rasterData\LC08_L2SP_146040_20241018_20241028_02_T1_SR_B7.TIF"
]

# Path to save the clipped rasters
output_dir = r"G:\Manjar\github\rasterData\clipBands"

# Read the shapefile
gdf = gpd.read_file(roi)

# Loop over each raster and clip it
for raster_path in rasters_path:
    # Construct the output path for each clipped raster
    clipped_path = os.path.join(output_dir, os.path.basename(raster_path).replace('.TIF', '_clipped.tif'))

    with rio.open(raster_path) as src:
        # Get the shapes from the shapefile
        shapes = gdf.geometry
        
        # Perform the clipping operation
        cropped_image, cropped_image_transform = rmask(src, shapes, crop=True)

        # Update the metadata of the clipped image
        out_meta = src.meta.copy()
        out_meta.update({
            "height": cropped_image.shape[1],
            "width": cropped_image.shape[2],
            "transform": cropped_image_transform
        })

        # Save the clipped raster to the output directory
        with rio.open(clipped_path, "w", **out_meta) as dst:
            dst.write(cropped_image)
